In [1]:
import os
def homePath(path):
    if path[0] == "~":
        return os.path.join(os.path.expanduser("~"), path.strip("~/"))
    else:
        return path

#Model Creation 1
#Figure out how to do versioning effectively
individualInput = False
combinedInputPath = "/home/jaredrussell/CPPMLGen"
#combinedInputPath = "/home/jaredrussell/CPPMLGenMini"
pathToTokenizedData = "C:\\Users\\mcall\\OneDrive\\Desktop\\DummyOutput\\Tokenizer\\"
pathToGradeData = "C:\\Users\\mcall\\OneDrive\\Desktop\\DummyOutput\\Grader\\"
GradesTokensName = ""


if not individualInput:
    pathToTokenizedData = os.path.join(combinedInputPath, "Tokens/")
    pathToGradeData = os.path.join(combinedInputPath, "Grades/")

if GradesTokensName == "":
    #Use newest folder for each

    #Get the newest folder for the tokens
    tokensFolders = os.listdir(pathToTokenizedData)
    tokensFolders.sort()
    pathToTokenizedData = os.path.join(pathToTokenizedData , tokensFolders[-1])

    #Get the newest folder for the grades
    gradesFolders = os.listdir(pathToGradeData)
    gradesFolders.sort()
    pathToGradeData = os.path.join(pathToGradeData,  gradesFolders[-1])
else:
    pathToTokenizedData = os.path.join(pathToTokenizedData, GradesTokensName)
    pathToGradeData = os.path.join(pathToGradeData, GradesTokensName)





modelOutputPath = "/home/jaredrussell/CPPMLGen/Models"



In [2]:
import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import tokenizer_from_json
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU, Dropout
from keras.callbacks import EarlyStopping
from keras.models import load_model
from keras.models import model_from_json
from keras.models import Model
from matplotlib import pyplot as plt
import tensorflow as tf
import keras
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import tokenizer_from_json
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
import keras.layers as layers
from keras.callbacks import EarlyStopping
from keras.models import load_model
from keras.models import model_from_json
from keras.models import Model




from keras.utils import pad_sequences
import os
import sys

sys.path.append('./SCA-Tokenizer/')
import CLPTokenizer as CLPTokenizer



#Load the data
#tokenized data is in tokenizedData.pkl, has tokenizer obj in tokenizer.json

#Load the tokenizer
with open(pathToTokenizedData + "/tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f, encoding='latin1')


#Load the tokenized data
with open(pathToTokenizedData + "/tokenizedData.pkl", "rb") as f:
    tokenizedData = pickle.load(f)

print (tokenizedData)

#Load the grade data
#gradeData is a dict with keys as the file names and values as the grades
with open(pathToGradeData + "/grades.pkl", "rb") as f:
    gradeData = pickle.load(f)

print (gradeData)

#load the group data
#with open(pathToTokenizedData + "/tokenGroupDataframe.pkl", "rb") as f:
#    tokenizedGroupData = pickle.load(f)

combinedDF = pd.merge(tokenizedData, gradeData, on = "Path")
#combinedDF = pd.merge(combinedDF, tokenizedGroupData, on = "Path")
print (combinedDF)







2023-11-29 16:14:33.742402: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-29 16:14:33.775775: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-29 16:14:33.775802: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-29 16:14:33.776718: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-29 16:14:33.782191: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-29 16:14:33.782743: I tensorflow/core/platform/cpu_feature_guard.cc:1

                                                tokenCode  \
1        8 8 8 8 280 2 489 1 8 280 2 489 1 8 8 138 405...   
4        8 8 8 8 280 2 489 1 8 280 2 489 1 8 8 138 405...   
27       165 12 12 12 280 2 118 1 2 1 12 280 2 118 1 2...   
28       28 28 28 28 28 28 28 28 28 156 18 2 11 42 1 5...   
29       165 12 12 12 280 2 118 1 2 1 12 280 2 118 1 2...   
...                                                   ...   
117158   45 19 718 280 2 489 1 138 30 359 496 3 302 15...   
117159   45 19 718 280 2 489 1 41 138 30 359 496 3 302...   
117161   45 19 718 280 2 489 1 41 138 30 359 496 3 302...   
117163   28 8 15 8 358 149 192 51 306 8 50 3 28 28 2 1...   
117170   28 28 45 2 1 261 209 150 405 47 83 97 526 86 ...   

                                                     Path  
1                                3385567/RBHook.cpp/0.cpp  
4                                3385567/RBHook.cpp/1.cpp  
27         3265841/app/src/main/cpp/passman-lib.cpp/4.cpp  
28         3265841/app/src/

In [12]:
#Padding

maxLen = 500
minLen = 100
#get rid of the ones that are too long

print (type(combinedDF["tokenCode"]))
combinedDF = combinedDF[combinedDF["tokenCode"].apply(lambda x: len(x)) <= maxLen -1]

#shorten the ones that are too long FOR TESTING
#combinedDF["tokenCode"] = combinedDF["tokenCode"].apply(lambda x: [int(i) for i in x.split()[:maxLen]])

#get rid of the ones that are too short DISABLED FOR TESTING
#combinedDF = combinedDF[combinedDF["tokenCode"].apply(lambda x: len(x)) > minLen]

#Pad the sequences
combinedDF["tokenCode"] = combinedDF["tokenCode"].apply(lambda x: [int(i) for i in x.split()]).tolist()
combinedDF["tokenCode"] = pad_sequences(combinedDF["tokenCode"].tolist(), maxlen=maxLen, padding="post", truncating="post").tolist()
#combinedDF["tokenGroupCode"] = pad_sequences(combinedDF["tokenGroupCode"], maxlen = maxLen, padding = "post", truncating = "post").tolist()

print(combinedDF)

#only use 5% of the data
#combinedDF = combinedDF.sample(frac=0.001, random_state=1)

#48590 



<class 'pandas.core.series.Series'>


ValueError: invalid literal for int() with base 10: ' 28 28 28 28 28 28 28 28 28 156 18 2 11 42 1 5 8 775 159 538 386 118 4 2 1 3 7 2 52 17 118 1 5 605 358 118 4 2 1 3 6 2 118 315 1 3 6 123 142 5 2 212 1 5 55 4 2 9 1 3 55 4 2 212 9 1 3 4 2 2 11 40 42 1

In [5]:
print (combinedDF)


                                               tokenCode  \
3       28 28 28 28 28 28 28 28 28 156 18 2 11 42 1 5...   
11      8 280 2 118 1 120 8 280 2 118 1 120 8 396 15 ...   
12      8 280 2 118 1 120 8 396 15 496 398 15 782 8 8...   
14      8 280 2 118 1 120 8 280 2 118 1 120 8 396 15 ...   
17      8 280 2 118 1 120 8 280 2 118 1 120 8 396 15 ...   
...                                                  ...   
48409   45 19 718 280 2 489 1 41 39 381 280 2 489 1 1...   
48417   28 18 2 11 358 1 8 8 8 8 8 8 8 5 63 42 3 3 2 ...   
48426   28 8 15 8 358 149 192 51 306 8 50 3 28 28 28 ...   
48430   45 12 47 19 324 86 12 267 114 19 12 114 47 19...   
48431   28 8 15 8 358 149 192 51 306 8 50 3 28 28 28 ...   

                                                    Path  fileGrade  
3         3265841/app/src/main/cpp/passman-lib.cpp/0.cpp  40.400000  
11         3378387/src/zmq/zmqabstractnotifier.cpp/0.cpp  25.600000  
12         3378387/src/zmq/zmqabstractnotifier.cpp/2.cpp  38.933333  

In [4]:
#if (not gpu_detected):
#    print("GPU not detected, using CPU")

number_of_tokens = combinedDF["tokenCode"].apply(lambda x: len(x)).max()
print (number_of_tokens)

model = Sequential()
model.add(Embedding(number_of_tokens, 64, input_length=maxLen))
model.add(layers.Conv1D(64, 3, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Reshape((1, 64)))
model.add(layers.Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.38479930887149405))
model.add(layers.Bidirectional(LSTM(32)))
model.add(layers.Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(loss='mean_squared_logarithmic_error', optimizer='Adam', metrics=['mse', 'mae', 'mape', 'accuracy'])





# convert inputs to numpy arrays
token_code = np.array(combinedDF["tokenCode"].tolist())
#token_group_code = np.array(combinedDF["tokenGroupCode"].tolist())
file_grade = np.array(combinedDF["fileGrade"].tolist())

hist = model.fit(token_code, file_grade, epochs=100, batch_size=32, verbose=1)


#Save the model in timestamp folder and with tokenizer
timestamp = str(pd.Timestamp.now()).replace(" ", "_").replace(":", "-").replace(".", "-")
if not os.path.exists(modelOutputPath):
    os.mkdir(modelOutputPath)
if not os.path.exists(modelOutputPath + "/" + timestamp):
    os.mkdir(modelOutputPath + "/" + timestamp)
model.save(modelOutputPath + "/" + timestamp + "/model.h5")
with open(modelOutputPath + "/" + timestamp + "/tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)
#make an archaive of the SCA-Tokenizer Folder
#get CWD
cwd = os.getcwd()
if os.path.exists(os.path.join(cwd, "SCA-Tokenizer")):
    os.system("tar -czvf \"" + modelOutputPath + "/" + timestamp + "/SCA-Tokenizer.tar.gz\" " + os.path.join(cwd, "SCA-Tokenizer"))

#Save the AutoGrader Folder
autoGraderDir = os.path.join(cwd, "../Auto-Grader/")
if os.path.exists(autoGraderDir):
    os.system("tar -czvf \"" + modelOutputPath + "/" + timestamp + "/Auto-Grader.tar.gz\" " + autoGraderDir)

#Save tokenizedGroupData
#with open(modelOutputPath + "/" + timestamp + "/tokenizedGroupDataframe.pkl", "wb") as f:
#    pickle.dump(tokenizedGroupData, f)

500
Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 5.0923 - mse: 198.4853 - mae: 10.5346 - mape: 237599.4531 - accuracy: 0.1667
Epoch 2/100
1/1 [==============================] - 0s 10ms/step - loss: 5.0614 - mse: 198.2984 - mae: 10.5277 - mape: 1296218.1250 - accuracy: 0.1667
Epoch 3/100
1/1 [==============================] - 0s 10ms/step - loss: 5.0220 - mse: 198.0753 - mae: 10.5215 - mape: 3719304.0000 - accuracy: 0.1667
Epoch 4/100
1/1 [==============================] - 0s 10ms/step - loss: 4.9783 - mse: 197.8449 - mae: 10.5129 - mape: 5437255.5000 - accuracy: 0.1667
Epoch 5/100
1/1 [==============================] - 0s 9ms/step - loss: 4.9209 - mse: 197.4838 - mae: 10.5027 - mape: 7042212.5000 - accuracy: 0.1667
Epoch 6/100
1/1 [==============================] - 0s 10ms/step - loss: 4.8608 - mse: 197.1545 - mae: 10.4890 - mape: 8628477.0000 - accuracy: 0.1667
Epoch 7/100
1/1 [==============================] - 0s 10ms/step - loss: 4.8247 - mse: 196.9326 - mae

/home/jaredrussell/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
tar: Removing leading `/' from member names
tar: Removing leading `/home/jaredrussell/gitRepos/SCA-ML-grade/Model-Generation/../' from member names
